In [65]:
import pandas as pd
import openpyxl
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import Alignment
import time
import numpy as np

In [66]:
def combine(x):
    # Функция для группировки всех значений в строку разделенную ;
    return  ';'.join(x)

def check_unique(x):
    # Функция для нахождения разночтений в площади выделенного гектара
    # создаем список разделяя по точке с запятой
    temp_lst = x.split(';')
    # Создаем множество оставляя только уникальные значения
    temp_set = set(temp_lst)
    if'nan' in temp_set:
        return 'Не заполнены значения площади лесотаксационного выдела!!!'
    else:
        return 'Площади совпадают' if len(temp_set) == 1 else 'Ошибка!!! Площади лесотаксационного выдела не совпадают'

def main_check_unique(x):
    # Функция для проверки корректности заполнения площади выдела
    temp_str = ';'.join(x) # Склеиваем все значения
    temp_lst = temp_str.split(';') # Создаем список разбивая по ;
    temp_set = set(temp_lst) # Превращаем в множество
    if len(temp_set) > 1: # Если длина множества больше 1 то есть погрешности
        return 0
    elif'nan' in temp_set:# если есть нан то не заполнены площади выдела
        return 0
    else:# Если все в порядке то возвращаем единственный элемент списка
        return convert_to_float(temp_lst[0])

def convert_to_float(x):
    """
    Функция для конвертирования строки в флоат.
    :param x: строка
    :return:
    """
    # очищаем от пробелов и возможных заполнения ячейки пробелом

    
    try:
        return float(x)
    except ValueError:
        return 0

In [67]:
file_data_xlsx = 'data/2022-10-27_64_Реестр УПП с дополнительными колонками..xlsx'
path_to_end_folder = 'data/'

In [68]:
df = pd.read_excel(file_data_xlsx,skiprows=8)

In [69]:
df

,1,2,3,4,5,6,7,8,9,10,...,28,29,30,31,32,33,34,35,36,37
0,Ангоянское,Ангоянское,NaN,32,1,13,1а,0.4,"56,8906071731145","111,604523118265",...,0,0,0.0,0,0.0,NaN,0.4,NaN,NaN,1.0
1,Ангоянское,Ангоянское,NaN,32,2,7,2а,1.6,"56,8907756254041","111,60655696686",...,0,0,0.0,0,0.0,NaN,1.6,NaN,NaN,1.0
2,Ангоянское,Ангоянское,NaN,32,3,11,3а,0.1,"56,8853268727696","111,611572021886",...,0,0,0.0,0,0.0,NaN,0.1,NaN,NaN,1.0
3,Ангоянское,Ангоянское,NaN,32,5,10,5а,7.5,"56,8885191270788","111,601674714733",...,0,0,0.0,0,0.0,NaN,7.5,NaN,NaN,1.0
4,Ангоянское,Ангоянское,NaN,32,7,78,7а,20.5,"56,8783556985033","111,572764198335",...,0,0,0.0,0,0.0,NaN,20.5,NaN,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214628,Хоринское,Хоринское сельское,Технический участок № 9 СПК «Верхне-Талецкий»,43,13,25,2,5.2,NaN,NaN,...,0,0,0.0,0,NaN,NaN,5.2,NaN,БУФ,1.0
214629,Хоринское,Хоринское сельское,Технический участок № 9 СПК «Верхне-Талецкий»,43,16,"5,4",16а,2.2,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,370,2.2,NaN,БУФ,1.0
214630,Хоринское,Хоринское сельское,Технический участок № 9 СПК «Верхне-Талецкий»,43,17,"6,5",17а,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,370,1.0,NaN,БУФ,1.0
214631,Хоринское,Хоринское сельское,Технический участок № 9 СПК «Верхне-Талецкий»,43,18,3,18а,2.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,370,2.0,NaN,БУФ,1.0


In [70]:
# Приводим названия колонок к строковому виду, чтобы избежать возможных проблем с названиями колонок
df.columns = list(map(str,list(df.columns)))
# на всякий случай очищаем от пробельных символов
df.columns = list(map(lambda x:x.replace(" ",""),list(df.columns)))

In [71]:
"""
Соответствие названий колонок используемым в программе номерам колонок
Лесничество -1
Участковое лесничество- 2
Урочище - 3
Номер лесного квартала -4
Номер лесотаксационного выдела -5
Площадь лесотаксационного выдела, га - 6
Площадь лесотаксационного выдела или его части (лесопатологического выдела), га - 8



"""

'\nСоответствие названий колонок используемым в программе номерам колонок\nЛесничество -1\nУчастковое лесничество- 2\nУрочище - 3\nНомер лесного квартала -4\nНомер лесотаксационного выдела -5\nПлощадь лесотаксационного выдела, га - 6\nПлощадь лесотаксационного выдела или его части (лесопатологического выдела), га - 8\n\n\n\n'

In [72]:
df.rename(columns={'1':'Лесничество','2':'Участковое лесничество','3':'Урочище','4':'Номер лесного квартала','5':'Номер лесотаксационного выдела',
                  '6':'Площадь лесотаксационного выдела, га','8':'Площадь лесотаксационного выдела или его части (лесопатологического выдела), га',},inplace=True)

In [73]:
# заполняем пропущенные места
df['Урочище'] = df['Урочище'].fillna('Название урочища не заполнено')

In [74]:
# СОздаем проверочный файл для проверки правильности ввода плошади выдела
check_df = df.copy(deep=True)

# Меняем тип столбца на строку чтобы создать строку включающую в себя все значения разделенные ;
check_df['Площадь лесотаксационного выдела, га'] = check_df['Площадь лесотаксационного выдела, га'].astype(str)
#

checked_pl = check_df.groupby(['Лесничество', 'Участковое лесничество', 'Урочище', 'Номер лесного квартала',
                               'Номер лесотаксационного выдела']).agg(
    {'Площадь лесотаксационного выдела, га': combine})

# Применяем функцию првоеряющую количество уникальных значений в столбце, если больше одного то значит есть ошибка в данных
checked_pl['Контроль совпадения площади выдела'] = checked_pl['Площадь лесотаксационного выдела, га'].apply(
    check_unique)

# переименовывам колонку
checked_pl.rename(columns={'Площадь лесотаксационного выдела, га': 'Все значения площади для указанного выдела'},
                  inplace=True)
# Извлекаем индексы в колонки
checked_pl = checked_pl.reset_index()

# Получаем текущую дату
current_time = time.strftime('%H_%M_%S %d.%m.%Y')
# Сохраняем отчет
# Для того чтобы увеличить ширину колонок для удобства чтения используем openpyxl
wb = openpyxl.Workbook() # Создаем объект
# Записываем результаты
for row in dataframe_to_rows(checked_pl,index=False,header=True):
    wb['Sheet'].append(row)

# Форматирование итоговой таблицы
# Ширина колонок
wb['Sheet'].column_dimensions['A'].width = 15
wb['Sheet'].column_dimensions['B'].width = 20
wb['Sheet'].column_dimensions['C'].width = 36
wb['Sheet'].column_dimensions['F'].width = 15
wb['Sheet'].column_dimensions['G'].width = 15
# Перенос строк для заголовков
wb['Sheet']['D1'].alignment = Alignment(wrap_text=True)
wb['Sheet']['E1'].alignment = Alignment(wrap_text=True)
wb['Sheet']['F1'].alignment = Alignment(wrap_text=True)
wb['Sheet']['G1'].alignment = Alignment(wrap_text=True)
wb['Sheet']['H1'].alignment = Alignment(wrap_text=True)


wb.save(f'{path_to_end_folder}/Проверка правильности ввода площадей лесотаксационного выдела {current_time}.xlsx')

In [75]:
# Основной отчет
# Готовим колонки к группировке
df['Площадь лесотаксационного выдела, га'] = df['Площадь лесотаксационного выдела, га'].astype(str)

df['Площадь лесотаксационного выдела, га'] = df['Площадь лесотаксационного выдела, га'].apply(
    lambda x: x.replace(',', '.'))

df['Площадь лесотаксационного выдела или его части (лесопатологического выдела), га'] = df[
    'Площадь лесотаксационного выдела или его части (лесопатологического выдела), га'].astype(str)

df['Площадь лесотаксационного выдела или его части (лесопатологического выдела), га'] = df[
    'Площадь лесотаксационного выдела или его части (лесопатологического выдела), га'].apply(
    lambda x: x.replace(',', '.'))


df['Площадь лесотаксационного выдела или его части (лесопатологического выдела), га'] = df[
    'Площадь лесотаксационного выдела или его части (лесопатологического выдела), га'].apply(convert_to_float)



In [76]:
df.head()

,Лесничество,Участковое лесничество,Урочище,Номер лесного квартала,Номер лесотаксационного выдела,"Площадь лесотаксационного выдела, га",7,"Площадь лесотаксационного выдела или его части (лесопатологического выдела), га",9,10,...,28,29,30,31,32,33,34,35,36,37
0,Ангоянское,Ангоянское,Название урочища не заполнено,32,1,13,1а,0.4,"56,8906071731145","111,604523118265",...,0,0,0.0,0,0.0,NaN,0.4,NaN,NaN,1.0
1,Ангоянское,Ангоянское,Название урочища не заполнено,32,2,7,2а,1.6,"56,8907756254041","111,60655696686",...,0,0,0.0,0,0.0,NaN,1.6,NaN,NaN,1.0
2,Ангоянское,Ангоянское,Название урочища не заполнено,32,3,11,3а,0.1,"56,8853268727696","111,611572021886",...,0,0,0.0,0,0.0,NaN,0.1,NaN,NaN,1.0
3,Ангоянское,Ангоянское,Название урочища не заполнено,32,5,10,5а,7.5,"56,8885191270788","111,601674714733",...,0,0,0.0,0,0.0,NaN,7.5,NaN,NaN,1.0
4,Ангоянское,Ангоянское,Название урочища не заполнено,32,7,78,7а,20.5,"56,8783556985033","111,572764198335",...,0,0,0.0,0,0.0,NaN,20.5,NaN,NaN,1.0


In [77]:
# Группируем
group_df = df.groupby(['Лесничество', 'Участковое лесничество', 'Урочище', 'Номер лесного квартала',
                       'Номер лесотаксационного выдела']).agg({'Площадь лесотаксационного выдела, га': main_check_unique,
                                                               'Площадь лесотаксационного выдела или его части (лесопатологического выдела), га': 'sum'})

In [78]:
group_df=group_df.astype(str)

In [79]:
# переименовываем колонку
group_df.rename(columns={
    'Площадь лесотаксационного выдела или его части (лесопатологического выдела), га': 'Используемая площадь лесотаксационного выдела, га'},
                inplace=True)

# Извлекаем индексы в колонки
group_df = group_df.reset_index()

group_df['Площадь лесотаксационного выдела, га'] = group_df['Площадь лесотаксационного выдела, га'].apply(convert_to_float)
group_df['Используемая площадь лесотаксационного выдела, га'] =group_df['Используемая площадь лесотаксационного выдела, га'].apply(convert_to_float)

In [80]:
# Округляем до 3 знаков для корректного сравнения
group_df['Площадь лесотаксационного выдела, га'] = np.round(group_df['Площадь лесотаксационного выдела, га'],decimals=3)
group_df['Используемая площадь лесотаксационного выдела, га'] = np.round(group_df['Используемая площадь лесотаксационного выдела, га'],decimals=3)


In [81]:
# Создаем колонку для контроля 
group_df['Контроль площади используемого надела'] = group_df['Площадь лесотаксационного выдела, га'] < group_df[
    'Используемая площадь лесотаксационного выдела, га']

group_df['Контроль площади используемого надела'] = group_df['Контроль площади используемого надела'].apply(
    lambda x: 'Превышение используемой площади выдела!!!' if x is True else 'Все в порядке')

In [82]:
# Изменяем состояние колонки если площадь всего выдела равна 0
group_df['Контроль правильности ввода площади лесотаксационного выдела'] = group_df['Площадь лесотаксационного выдела, га'].apply(
    lambda x: 'Площадь лесотаксационного выдела равна нулю или  обнаружены разные значения площади выдела !!!' if x == 0 else 'Площади лесотаксационного выдела не отличаются друг от друга')

In [83]:
# Сохраняем отчет
# Для того чтобы увеличить ширину колонок для удобства чтения используем openpyxl
wb = openpyxl.Workbook() # Создаем объект
# Записываем результаты
for row in dataframe_to_rows(group_df,index=False,header=True):
    wb['Sheet'].append(row)

# Форматирование итоговой таблицы
# Ширина колонок
wb['Sheet'].column_dimensions['A'].width = 15
wb['Sheet'].column_dimensions['B'].width = 20
wb['Sheet'].column_dimensions['C'].width = 36
wb['Sheet'].column_dimensions['F'].width = 15
wb['Sheet'].column_dimensions['G'].width = 15
wb['Sheet'].column_dimensions['H'].width = 20
# Перенос строк для заголовков
wb['Sheet']['D1'].alignment = Alignment(wrap_text=True)
wb['Sheet']['E1'].alignment = Alignment(wrap_text=True)
wb['Sheet']['F1'].alignment = Alignment(wrap_text=True)
wb['Sheet']['G1'].alignment = Alignment(wrap_text=True)
wb['Sheet']['H1'].alignment = Alignment(wrap_text=True)

wb.save(f'{path_to_end_folder}/Контроль используемых площадей лесотаксационных выделов от {current_time}.xlsx')